# Auto generated report for France

This notebook generates for each available geographic zone a summary of the most recent simulation.


## Preparation

### Loading libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display, Markdown, Latex

In [3]:
import os
from functools import partial

In [4]:
from matplotlib import pyplot as plt
import matplotlib
import pandas as pd
from numpy import unique

In [5]:
from model_analysis import * 

### Loading data

In [6]:
run_manifest = read_manifest('run-manifest.csv')
analysis_set = get_analysis_set(run_manifest)

In [7]:
def add_to_zone_dict(zone_dict, row, i, zone):
    zone_dict[zone] = {
        "index": i,
        "date": row["last available data"],
        "version": row["version"],
    }

zone_dict = {}
for i, row in analysis_set.iterrows():
    for zone in row["modelling zones"]:
        if zone not in zone_dict:
            add_to_zone_dict(zone_dict, row, i, zone)
        elif (zone_dict[zone]["date"] < row["last available data"])\
            or (zone_dict[zone]["date"] == row["last available data"]
                and zone_dict[zone]["version"] < row["version"]):
            add_to_zone_dict(zone_dict, row, i, zone)
        

In [8]:
indices = unique([zone_dict[d]['index'] for d in zone_dict])
country_2_region = {}
for idx in indices:
    for country in analysis_set.loc[idx,].model.data['ifr']['country'].unique():
        country_2_region[country] = {}

    for _, row in analysis_set.loc[idx,].model.data['ifr'].iterrows():
        country_2_region[row['country']][row['region']] = zone_dict[row['region']]['index']


## Plotting functions

First we define a custom process that will print for each zone.

In [9]:
def process_zone(zone, zone_dict, model_data, title_ext=""):
    display(Markdown(
        f"### {zone}{title_ext} \n\n Simulation sur la base de donnees du {zone_dict['date']}"
        + f" avec la version {zone_dict['version']} du code d'Imperial College a Londres"
    ))
    axs = plot_zones_summary(zone, model_data)
    axis_date_limits(axs, max_date='2020-05-20')
    translate_axes_to_french(axs)
    plt.show()
    plt.close()

In [14]:
def print_zone(file_in, img_dir, file_dir, zone, zone_dict, model_data, title_ext="", img_ext=".png"):
    img_file = os.path.join(img_dir, (zone + '_' + zone_dict['date'].strip() + img_ext).replace(" ", "_"))
    
    unix_rel_img_path = os.path.relpath(img_file, file_dir).replace('\\','/')
    file_in.write(
        f"### {zone}{title_ext} \n\n Simulation sur la base de données du {zone_dict['date']}"
        + f" avec la version {zone_dict['version']} du code d'Imperial College à Londres"
        +  f" \n\n ![img_file]({unix_rel_img_path})\n\n"
    )
    axs = plot_zones_summary(zone, model_data)
    axis_date_limits(axs, max_date='2020-05-20')
    translate_axes_to_french(axs)
    axs[0].figure.savefig(img_file, bbox_inches='tight')
    plt.close(axs[0].figure)

And then we apply this function to every zone that has been identified.

In [18]:
def apply_process_to_France(
    process_func, country_2_region, analysis_set, zone_dict,
    display_func=lambda x:display(Markdown(x))):
    country = "France"
    display_func(f"# Rapport de simulation sur la progression et l'impact du COViD19 en {country}\n\n")

    display_func(
        "Ce rapport présente les resultats du travail de réplication d'Imperial College sur la progression de l'épidémie COVID-19 en Europe. Ce travail, reprend les mêmes méthodes et les appliquent à la France et ses régions.\n\n" +
        "Toutes les régions françaises sont passées au crible. À chaque fois trois graphiques illustrent la progression du nombre de décès, un quatrième montre l'effondrement du taux de reproduction initial et le dernier extrapole la progression du nombre de cas confirmés, à partir d'un taux de mortalité estimé à 1,12% (scénario plutôt pessimiste).\n\n" +
        "D'une manière générale le projet vise à estimer au plus près l'impact de cinq interventions politiques (distanciation physique, confinement, annulation des grands événements, fermeture des écoles et université, mise en quarantaine des personnes infectée) sur la courbe des décès liés à la pandémie. En ce sens, les données de 'Google Community Reports' sont en cours d'intégration afin d'améliorer la capacité prédictive du modèle.\n\n" +
        "Les prédictions presentées dans les graphes ci-dessous (lignes pointillées) ne prennent pas en compte le déconfinement. Elles ne sont présentées ici que pour accentuer les tendances observées durant le confinement.\n\n"
    )

    
    display_func(f"## Résultats au niveau du Pays\n\n")
    display_func(
        "Dans cette section vous retrouverez des délimitations non géographiques"
        + "de la population, nottament une modelisation sur la base des données:\n\n"
        + "- Santé publique France (SPF) via le ECDC sous la dénomination 'France'\n"
        + "- SPF via la collecte de opencovid19-fr sous la dénomination 'France-OC19'\n"
        + "- Des décès en EHPAD sous la dénomination 'France-EHPAD'\n"
        + "- Des décès en hopitaux sous la dénomination 'France-hopitaux'\n\n"
    )
    country_level_list = []
    region_level_list = []
    for region in country_2_region[country]:
        if region[:len(country)] == country:
            country_level_list.append(region)
        else:
            region_level_list.append(region)

    for zone in sorted(country_level_list):
        idx = zone_dict[zone]['index']
        process_func(
            zone, zone_dict[zone], 
            analysis_set.loc[idx, 'model'].data)

    display_func(f"## Résultats régionaux \n")
    for zone in sorted(region_level_list):
        idx = zone_dict[zone]['index']
        process_func(
            zone, zone_dict[zone], 
            analysis_set.loc[idx, 'model'].data)


In [12]:
#apply_process_to_France(process_zone, country_2_region, analysis_set, zone_dict)

In [19]:

report_dir = os.path.join('public_reports', 'france_report')
img_dir = os.path.join(report_dir, 'img')
os.makedirs(report_dir, exist_ok=True)
os.makedirs(img_dir, exist_ok=True)
report_file = os.path.join(report_dir, "france_report.md")
with open(report_file, "w", encoding="utf-8") as file_in:
    print_partial = partial(print_zone, file_in, img_dir, report_dir)
    apply_process_to_France(
        print_partial, country_2_region, analysis_set, zone_dict,
        file_in.write)